<a href="https://colab.research.google.com/github/MatthewMair42/migration/blob/main/MIMI_data_migr_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data extraction on MIMI dataset

**Import libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import holoviews as hv
import statistics
from scipy.stats import kendalltau, pearsonr, spearmanr
import seaborn as sns
import matplotlib.patches as mpatches
import math

**Pandas display options**

In [ ]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

### Download dataset

Retrieve dataset from Zenodo and read the CSV file:

In [ ]:
path = 'https://zenodo.org/record/6493325/files/mimi_dataset_v2.csv?download=1'

In [ ]:
mimi = pd.read_csv(path, index_col=[0], sep=",", low_memory=False)

# MM work


# Reshaping


In [ ]:
columns_to_keep = ['origin_country', 'destination_country',
                   'UN_migr_stocks_2000_T_T', 'UN_migr_stocks_2005_T_T', 'UN_migr_stocks_2010_T_T',
                   'UN_migr_stocks_2015_T_T', 'UN_migr_stocks_2020_T_T', 'UN_migr_stocks_2000_M_T',
                   'UN_migr_stocks_2005_M_T', 'UN_migr_stocks_2010_M_T', 'UN_migr_stocks_2015_M_T',
                   'UN_migr_stocks_2020_M_T', 'UN_migr_stocks_2000_F_T', 'UN_migr_stocks_2005_F_T',
                   'UN_migr_stocks_2010_F_T', 'UN_migr_stocks_2015_F_T', 'UN_migr_stocks_2020_F_T']

# Create the subset copy
mimi_subset = mimi[columns_to_keep].copy()

# Display the first few rows of the subset
mimi_subset.head(5)

,origin_country,destination_country,UN_migr_stocks_2000_T_T,UN_migr_stocks_2005_T_T,UN_migr_stocks_2010_T_T,UN_migr_stocks_2015_T_T,UN_migr_stocks_2020_T_T,UN_migr_stocks_2000_M_T,UN_migr_stocks_2005_M_T,UN_migr_stocks_2010_M_T,UN_migr_stocks_2015_M_T,UN_migr_stocks_2020_M_T,UN_migr_stocks_2000_F_T,UN_migr_stocks_2005_F_T,UN_migr_stocks_2010_F_T,UN_migr_stocks_2015_F_T,UN_migr_stocks_2020_F_T
from_to,,,,,,,,,,,,,,,,,
AD-AR,AD,AR,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
AD-AT,AD,AT,2.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,3.0,NaN,NaN,NaN,1.0,NaN
AD-AU,AD,AU,30.0,30.0,20.0,20.0,30.0,15.0,15.0,10.0,12.0,17.0,15.0,15.0,10.0,8.0,13.0
AD-AW,AD,AW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AD-AX,AD,AX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Using the dataset named 'mimi_subset'
def reshape_migration_data(df=mimi_subset):
    # Step 1: Identify column patterns and create mapping
    years = [2000, 2005, 2010, 2015, 2020]
    categories = ['T_T', 'M_T', 'F_T']  # Total, Male, Female

    # Step 2: Create an empty list to store long-format data
    result_df = []

    # Step 3: Process each row (country pair) in the original dataframe
    for _, row in df.iterrows():
        origin = row['origin_country']
        destination = row['destination_country']

        # Process for each category (Total, Male, Female)
        for category in categories:
            # Extract values for specific years from the original data
            values_by_year = {}
            for year in years:
                col_name = f'UN_migr_stocks_{year}_{category}'
                values_by_year[year] = row[col_name]

            # Generate yearly data, filling in intermediate years with previous values
            for year in range(2000, 2021):
                # Find the appropriate reference year (the most recent year in our data that's <= current year)
                ref_year = max([y for y in years if y <= year])

                # Create a row for this year
                new_row = {
                    'origin_country': origin,
                    'destination_country': destination,
                    'year': year,
                    'category': category,
                    'migrant_stock': values_by_year[ref_year]
                }
                result_df.append(new_row)

    # Convert to dataframe
    result_df = pd.DataFrame(result_df)

    # Optionally, create separate columns for each category instead of a category column
    # This will make the data wider but easier to work with for some analyses
    pivot_df = result_df.pivot_table(
        index=['origin_country', 'destination_country', 'year'],
        columns='category',
        values='migrant_stock'
    ).reset_index()

    # Rename the columns to be more descriptive
    pivot_df.columns.name = None
    pivot_df = pivot_df.rename(columns={
        'T_T': 'total_migrants',
        'M_T': 'male_migrants',
        'F_T': 'female_migrants'
    })

    return result_df, pivot_df

# Execute the function with your mimi_subset dataset
long_format_df, pivot_format_df = reshape_migration_data()

# Now you have two dataframes:
# - long_format_df: fully long format with category as a column
# - pivot_format_df: semi-wide format with separate columns for total, male, and female migrants

# Example of how to view the first few rows
print("Long format sample:")
print(long_format_df.head())
print("\nPivot format sample:")
print(pivot_format_df.head())

Long format sample:
  origin_country destination_country  year category  migrant_stock
0             AD                  AR  2000      T_T            1.0
1             AD                  AR  2001      T_T            1.0
2             AD                  AR  2002      T_T            1.0
3             AD                  AR  2003      T_T            1.0
4             AD                  AR  2004      T_T            1.0

Pivot format sample:
  origin_country destination_country  year  female_migrants  male_migrants  \
0             AD                  AR  2000              NaN            1.0   
1             AD                  AR  2001              NaN            1.0   
2             AD                  AR  2002              NaN            1.0   
3             AD                  AR  2003              NaN            1.0   
4             AD                  AR  2004              NaN            1.0   

   total_migrants  
0             1.0  
1             1.0  
2             1.0  
3      

In [31]:
pivot_format_df.head(10)

,origin_country,destination_country,year,female_migrants,male_migrants,total_migrants
0,AD,AR,2000,NaN,1.0,1.0
1,AD,AR,2001,NaN,1.0,1.0
2,AD,AR,2002,NaN,1.0,1.0
3,AD,AR,2003,NaN,1.0,1.0
4,AD,AR,2004,NaN,1.0,1.0
5,AD,AR,2005,NaN,2.0,2.0
6,AD,AR,2006,NaN,2.0,2.0
7,AD,AR,2007,NaN,2.0,2.0
8,AD,AR,2008,NaN,2.0,2.0
9,AD,AR,2009,NaN,2.0,2.0


In [32]:
pivot_format_df = pivot_format_df.rename(columns={'female_migrants': 'migr_stocks_F'})
pivot_format_df = pivot_format_df.rename(columns={'male_migrants': 'migr_stocks_M'})
pivot_format_df = pivot_format_df.rename(columns={'total_migrants': 'migr_stocks_T'})

# Display the updated column names to verify
pivot_format_df.columns

Index(['origin_country', 'destination_country', 'year', 'migr_stocks_F',
       'migr_stocks_M', 'migr_stocks_T'],
      dtype='object')

In [33]:
import pandas as pd

# Assuming your dataframe is called 'df' with columns:
# 'year', 'origin', 'destination', 'migr_stocks_F', 'migr_stocks_M', 'migr_stocks_T'

# Create a reference dataframe with reversed country pairs
df_reversed = pivot_format_df.copy()
# Swap origin and destination
df_reversed['temp'] = df_reversed['origin_country']
df_reversed['origin_country'] = df_reversed['destination_country']
df_reversed['destination_country'] = df_reversed['temp']
df_reversed = df_reversed.drop('temp', axis=1)

# For clarity in the merge, rename the migrant stock columns in the reversed dataframe
df_reversed = df_reversed.rename(columns={
    'migr_stocks_F': 'origin_migr_stocks_F',
    'migr_stocks_M': 'origin_migr_stocks_M',
    'migr_stocks_T': 'origin_migr_stocks_T'
})

# Merge the original and reversed dataframes
result = pivot_format_df.merge(
    df_reversed[['year', 'origin_country', 'destination_country', 'origin_migr_stocks_F', 'origin_migr_stocks_M', 'origin_migr_stocks_T']],
    on=['year', 'origin_country', 'destination_country'],
    how='left'
)

In [34]:
result = result.rename(columns={'migr_stocks_F': 'dest_migr_stocks_F'})
result = result.rename(columns={'migr_stocks_M': 'dest_migr_stocks_M'})
result = result.rename(columns={'migr_stocks_T': 'dest_migr_stocks_T'})

In [35]:
result.head(50)

,origin_country,destination_country,year,dest_migr_stocks_F,dest_migr_stocks_M,dest_migr_stocks_T,origin_migr_stocks_F,origin_migr_stocks_M,origin_migr_stocks_T
0,AD,AR,2000,NaN,1.0,1.0,71.0,80.0,151.0
1,AD,AR,2001,NaN,1.0,1.0,71.0,80.0,151.0
2,AD,AR,2002,NaN,1.0,1.0,71.0,80.0,151.0
3,AD,AR,2003,NaN,1.0,1.0,71.0,80.0,151.0
4,AD,AR,2004,NaN,1.0,1.0,71.0,80.0,151.0
5,AD,AR,2005,NaN,2.0,2.0,233.0,261.0,494.0
6,AD,AR,2006,NaN,2.0,2.0,233.0,261.0,494.0
7,AD,AR,2007,NaN,2.0,2.0,233.0,261.0,494.0
8,AD,AR,2008,NaN,2.0,2.0,233.0,261.0,494.0
9,AD,AR,2009,NaN,2.0,2.0,233.0,261.0,494.0


In [36]:
UN_migr_stocks_pivot = result.copy()

# Now you can refer to the DataFrame using the new name:
UN_migr_stocks_pivot.head()

,origin_country,destination_country,year,dest_migr_stocks_F,dest_migr_stocks_M,dest_migr_stocks_T,origin_migr_stocks_F,origin_migr_stocks_M,origin_migr_stocks_T
0,AD,AR,2000,NaN,1.0,1.0,71.0,80.0,151.0
1,AD,AR,2001,NaN,1.0,1.0,71.0,80.0,151.0
2,AD,AR,2002,NaN,1.0,1.0,71.0,80.0,151.0
3,AD,AR,2003,NaN,1.0,1.0,71.0,80.0,151.0
4,AD,AR,2004,NaN,1.0,1.0,71.0,80.0,151.0


In [37]:
# Save to csv

UN_migr_stocks_pivot.to_csv('UN_migr_stocks_pivot.csv', index=False)